### CryoParticleSegment

In [ ]:
#%%bash
#git clone git+https://cyanazuki@github.com/cyanazuki/CryoParticleSegment.git
#cd CryoParticleSegment
#python setup.py install

In [ ]:
%pip install torchinfo -qq
%pip install -U git+https://github.com/qubvel/segmentation_models.pytorch -qq
%pip install starfile -qq
%pip install https://github.com/soft-matter/trackpy/archive/master.zip -qq
%pip install pycuda -qq

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 61.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.5/417.5 kB 28.2 MB/s eta 0:00:00
     \ 10.7 MB 11.1 MB/s 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 59.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.7/88.7 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.9 MB/s eta 0:00:00


In [ ]:
%pip install mrcfile -qq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.6 MB/s eta 0:00:00


## ⭐ Setup
You must run all codes under this category.

### ✅ Directory Settings

In [ ]:
# @title  { display-mode: "form" }

IMAGE_DIR = "/content/drive/MyDrive/research_xs/dataset/10081/processed_micrographs_np_split" # @param {type:"string"}
LABEL_DIR = "/content/drive/MyDrive/research_xs/dataset/10081/micrographs_ground_np" # @param {type:"string"}
RESULT_DIR = "/content/drive/MyDrive/research_xs/results/10081/unet_eb5_dice_postcrf" # @param {type:"string"}

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# @title  { display-mode: "form" }
# @markdown Detect whether using folder in Google Drive as **`RESULT DIR`**📁.

if "content" in IMAGE_DIR.split("/")[:3] or "content" in LABEL_DIR.split("/")[:3]:
  try:
    from google.colab import drive
    drive.mount('/content/drive')
    !rm -r /content/sample_data
    if "content" in IMAGE_DIR.split("/")[:3]:
      !cp -r {IMAGE_DIR} /content/image_dir
      IMAGE_DIR = "/content/image_dir"
    if "content" in LABEL_DIR.split("/")[:3]:
      !cp -r {LABEL_DIR} /content/label_dir
      LABEL_DIR = "/content/label_dir"
  except:
    pass

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
rm: cannot remove '/content/sample_data': No such file or directory


In [ ]:
IMAGE_DIR = "/content/image_dir"

In [ ]:
# @title  { display-mode: "form" }
# @markdown Source code directory.
SRC_DIR = "/content/drive/MyDrive/00_Image\ meeting/Researches/01_Scratchpad/seg_src/CryoParticleSegment-main/Modeling" # @param {type:"string"}

if True:
  !cp -rf {SRC_DIR}/* /content/
else:
  !cp {SRC_DIR}/convcrf.py /content/convcrf.py
  !cp {SRC_DIR}/dataset.py /content/dataset.py
  !cp {SRC_DIR}/lr_scheduler.py /content/lr_scheduler.py
  !cp {SRC_DIR}/metrics.py /content/metrics.py
  !cp {SRC_DIR}/model.py /content/model.py
  !cp {SRC_DIR}/trainer.py /content/trainer.py
  !cp {SRC_DIR}/utils.py /content/utils.py

### ✅ Packages Handling

In [ ]:
# @title  { display-mode: "form" }
# @markdown Useful packages.

import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import torch
from torch.utils.data import DataLoader

In [ ]:
# @title  { display-mode: "form" }
# @markdown User-defined packages.

from dataset import MicrographDataset, MicrographDatasetEvery

## ⭐ Main

### ✅ Setting

In [ ]:
# @markdown Parameters.

BATCH = 2
CROP_SIZE = (512, 512)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# @markdown Set seed.

random_state = 42
torch.manual_seed(random_state)
torch.cuda.manual_seed_all(random_state)

### ✅ Dataset

In [ ]:
train_dir = os.path.join(IMAGE_DIR, 'train')
train_filenames = np.loadtxt(f"{IMAGE_DIR}/train_filenames.txt", dtype=str)
train_dataset = MicrographDataset(image_dir=train_dir, label_dir=LABEL_DIR, filenames=train_filenames, crop_size=CROP_SIZE, num_patches = 4, crop=None)

In [ ]:
val_dir = os.path.join(IMAGE_DIR, 'val')
val_filenames = np.loadtxt(f"{IMAGE_DIR}/val_filenames.txt", dtype=str)
val_dataset = MicrographDatasetEvery(image_dir=val_dir, label_dir=LABEL_DIR, filenames=val_filenames, crop_size=CROP_SIZE, crop=None)
val_loader = DataLoader(val_dataset, batch_size=None, shuffle=False, pin_memory=True)

In [ ]:
test_dir = os.path.join(IMAGE_DIR, 'test')
test_filenames = np.loadtxt(f"{IMAGE_DIR}/test_filenames.txt", dtype=str)
test_dataset = MicrographDatasetEvery(image_dir=test_dir, label_dir=LABEL_DIR, filenames=test_filenames, crop_size=CROP_SIZE, crop=None)
test_loader = DataLoader(test_dataset, batch_size=None, shuffle=False, pin_memory=True)

## ⭐ Evaluate

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

from torchvision.utils import save_image
import starfile
import pandas as pd
import matplotlib
from PIL import Image
import cv2

def get_basename_with_uid_removed(path):
  return os.path.basename(path).split(sep='_', maxsplit=1)[-1]


def simple_micrograph_preprocessing(micrograph):
  micrograph_copy = micrograph.copy()
  micrograph_copy = (micrograph_copy-micrograph.mean()+2.5*micrograph.std())/5/micrograph.std()
  micrograph_copy[micrograph_copy<0]=0
  micrograph_copy[micrograph_copy>1]=1
  return micrograph_copy


In [ ]:
shape = None
for i1, i2, i3, i4 in val_loader: #test loader and reconstruct
    shape = i4.shape
    break

In [ ]:
# @title  { vertical-output: true, display-mode: "form" }
EMPIAR_ID = 10081 # @param {type:"integer"}
RADIUS = 77 # @param {type:"integer"}

In [ ]:
!cp /content/drive/MyDrive/research_xs/dataset/{EMPIAR_ID}/ground_truth/empiar-{EMPIAR_ID}_particles_selected.star .

In [ ]:
y_size = shape[1]
labeled_particles = starfile.read(f"empiar-{EMPIAR_ID}_particles_selected.star")['particles']
labeled_particles = labeled_particles[['rlnMicrographName', 'rlnCoordinateX', 'rlnCoordinateY']]
labeled_particles.columns = pd.Index(['image_name', 'x_coord', 'y_coord'])
labeled_particles['image_name'] = labeled_particles['image_name'].apply(get_basename_with_uid_removed)
labeled_particles['image_name'] = labeled_particles['image_name'].apply(lambda s: s.split(".")[0])
labeled_particles['y_coord'] = y_size - labeled_particles['y_coord']
labeled_particles

,image_name,x_coord,y_coord
0,HCN1apo_0183_2xaligned,2041,3454
1,HCN1apo_0183_2xaligned,985,1017
2,HCN1apo_0183_2xaligned,2258,1772
3,HCN1apo_0183_2xaligned,1529,3218
4,HCN1apo_0183_2xaligned,3416,3582
...,...,...,...
39347,HCN1apo_0686_2xaligned,1861,3704
39348,HCN1apo_0686_2xaligned,966,1126
39349,HCN1apo_0686_2xaligned,2316,991
39350,HCN1apo_0686_2xaligned,2047,2156


In [ ]:
def preprocess_and_crop(micrograph, crop_size=3968):
    processed_micrograph = simple_micrograph_preprocessing(micrograph)
    if crop_size:
        mic_width, mic_height = processed_micrograph.shape[1], processed_micrograph.shape[0]
        start_x, start_y = (mic_width - crop_size) // 2, (mic_height - crop_size) // 2
        end_x, end_y = start_x + crop_size, start_y + crop_size
        return processed_micrograph[start_y:end_y, start_x:end_x]
    else:
        return processed_micrograph

def plot_micrograph_and_labels(ax, micrograph, labels, coords):
    ax.imshow(micrograph, cmap='gray')
    ax.imshow(labels, cmap='gray', alpha=0.5)
    for x, y in coords:
        corrected_x, corrected_y = x, y
        circle = matplotlib.patches.Circle((corrected_x, corrected_y), radius=RADIUS, fill=False, color='r')
        ax.add_patch(circle)

In [ ]:
label_images = np.empty((0, shape[1], shape[2]), dtype=np.uint8)
gts = []

for idx, (test_image, _, grid, _) in enumerate(test_dataset):
    if idx == 6:
        break
    name = test_filenames[idx][:-4]
    micrograph = np.load(f"{IMAGE_DIR}/test/{name}.npy")
    label_path = f"{LABEL_DIR}/{name}.png"
    image = Image.open(label_path)
    label_image = np.array(image)

    cropped_micrograph = preprocess_and_crop(micrograph, crop_size=None)
    cropped_label_image = preprocess_and_crop(label_image, crop_size=None)

    label_images = np.concatenate((label_images, [cropped_label_image]), axis=0)

    locations = labeled_particles[labeled_particles['image_name'] == name]
    _, ax = plt.subplots(figsize=(12, 12))
    coords = locations[['x_coord', 'y_coord']].values # - 64 for 10017
    plot_micrograph_and_labels(ax, cropped_micrograph, cropped_label_image, coords)
    plt.show()
    print(len(coords))
    gts.append(coords)
    ##

#filename = f"{os.path.splitext(checkpoint_path)[0]}.png"
#pred_path = os.path.join(RESULT_DIR, "Each_ckpt", filename)
#save_image(pred_image, pred_path)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
label_images.shape

(6, 3838, 3710)

### CV approach

In [ ]:
radius = RADIUS

In [ ]:
import skimage as ski
from skimage import img_as_float
from scipy import ndimage as ndi
from skimage import filters, segmentation, morphology
import itertools


def normalize(im):
    max_mrc=np.max(im)
    min_mrc=np.min(im)
    img_original=(255*((im-min_mrc)/(max_mrc-min_mrc))).astype(np.uint8)
    return(img_original)

def min_rect_circle(cont):
    contours_poly = cv2.approxPolyDP(cont, 3, True)
    center, _= cv2.minEnclosingCircle(contours_poly)
    rect=cv2.minAreaRect(cont)
    box=np.int0(cv2.boxPoints(rect))
    mn,mx=np.amin(box,axis=0),np.amax(box,axis=0)
    diff=mx-mn
    if np.all(diff<(2*radius+40)):
        return(int(center[0]),int(center[1]))
    else:
        pass

def eliminate_near(fields):
    fields=np.array(fields,dtype=np.int32)
    i=np.arange(len(fields))
    diff=fields.reshape(len(fields),1,2)-fields
    D=np.sqrt((diff**2).sum(2))
    D=np.array(D,dtype=np.float64)
    D[np.triu_indices(D.shape[0])]=np.inf
    re = np.where(D< radius)
    out=np.array(list(zip(re[0],re[1])),dtype=np.int32)
    outmin=np.unique(np.min(out,axis=1))
    return(outmin)

def primary_sorting(i):
    #print("primary_sorting done")
    i1,i2,i3,i4=i[1]-(radius*2),i[0]-(radius*2),i[1]+(radius*2),i[0]+(radius*2)
    if i1<0 and i2<0:
        center=(i1+radius*2,i1+radius*2)
        i1=0
        i2=0
    elif i3 > thresh1.shape[0] and i4 > thresh1.shape[1]:
        i3= thresh1.shape[0]
        i4 = thresh1.shape[1]
        center=(radius*2,radius*2)
    elif i1<0:
        center=(i1+radius*2,i1+radius*2)
        i1=0
    elif i2<0:
        center=(i2+radius*2,i2+radius*2)
        i2=0
    elif i3 > thresh1.shape[0]:
        i3= thresh1.shape[0]
        center=(radius*2,radius*2)
    elif i4 > thresh1.shape[1]:
        i4 = thresh1.shape[1]
        center=(radius*2,radius*2)
    else:
        center=(radius*2,radius*2)


    th1=thresh1[i1:i3,i2:i4]

    c_frame=output_image[i1:i3,i2:i4]
    c_frame = cv2.cvtColor(c_frame, cv2.COLOR_GRAY2RGB)
    mask = np.zeros_like(th1)
    mask2 = np.zeros_like(th1)
    mask3 = np.zeros_like(c_frame)
    mask=cv2.circle(mask, center=center, radius=radius,color=(255),thickness=-1)
    mask2=cv2.circle(mask2, center=center, radius=radius+10,color=(255),thickness=-1)
    mask3=cv2.circle(mask3, center=center, radius=radius+10,color=[255,255,255],thickness=-1)
    col_mask= np.bitwise_and(c_frame,mask3)
    result = np.bitwise_and(th1,mask)
    result2 = np.bitwise_and(th1,mask2)
    col_a=np.where(np.all(col_mask== [0,255,0] , axis=-1))
    col_b=np.where(np.all(col_mask== [0,255,255] , axis=-1))
    inn_positions=np.nonzero(result)
    out_positions=np.nonzero(result2)

    contours, hierarchy = cv2.findContours(result,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    l=len(contours)
    a,b= (inn_positions[0].max()-inn_positions[0].min()),(inn_positions[1].max()-inn_positions[1].min())
    c,d= (out_positions[0].max()-out_positions[0].min()),(out_positions[1].max()-out_positions[1].min())
    if (len(col_a[0]) or len(col_b[0]))>0 :
        pass
    elif (c or d) > (radius*2.1):
        pass
    elif l==1 and ((c and d) <(radius*2)):
        return(tuple([i[0],i[1]]))
    else:

        #cv2.circle(frame, i, radius, (255, 0, 0), 3)
        ans={}
        centerx=range(0+radius,th1.shape[1]-radius,5)
        centery=range(0+radius,th1.shape[0]-radius,5)
        rad=range(radius,5,5)
        for cx,cy in itertools.product(centerx,centery):
            mask00 = np.zeros_like(th1)
            mask20=mask.copy()
            mask00=cv2.circle(mask00, center=(cx,cy), radius=radius, color=(255,255,255), thickness=-1)
            mask20=cv2.circle(mask20, center=(cx,cy), radius=radius+5, color=(255,255,255), thickness=-1)
            result00 = np.bitwise_and(th1,mask00)
            tot_ellip=np.sum(mask00 == [255])
            tot_ellip_white1=np.sum(result00 == [255])
            per_white=(tot_ellip_white1/tot_ellip)*100
            result002 = np.bitwise_and(th1,mask20)
            tot_ellip_white2=np.sum(result002 == [255])
            diff=tot_ellip_white2-tot_ellip_white1

            if diff<10 and per_white>10:
                ans[per_white]=(cx,cy)


        if len(ans.keys())>0:
            k=ans[max(ans.keys())]
            return (tuple([i2+k[0],i1+k[1]]))
        else:
            pass

In [ ]:
cv_list_all = []
e_factor = [2,4,6]
s_factor = [0.6,1,1.4]

for e in e_factor:
    for s in s_factor:
        cv_list = []
        print(f"e_factor: {e}, s_factor: {s}")
        for img in label_images:
            thresh1 = normalize(img)
            kernel_size = int(radius / 4)  # Example ratio
            kernel = np.ones((kernel_size, kernel_size), np.uint8)
            thresh1 = cv2.erode(thresh1, kernel, iterations=1)

            kernel_size = int(radius / e)  # Example ratio
            kernel = np.ones((kernel_size, kernel_size), np.uint8)
            thresh1 = cv2.erode(thresh1, kernel, iterations=1)

            contours, hierarchy = cv2.findContours(thresh1,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
            cont_array = [c for c in contours]

            # Filter out small/large region and find bounding box with center
            contr_min = radius**s
            c_ = np.array([cv2.contourArea(contour) for contour in contours])
            aa = (c_>contr_min) & (c_<500000)
            aa = aa.tolist()
            c_full_list = [d for d, keep in zip(cont_array, aa) if keep]
            c_list = (list(map(lambda x: min_rect_circle(x), c_full_list)))
            c_list = [x for x in c_list if x is not None]
            cv_list.append(c_list)
            print(len(c_), len(c_list))
        cv_list_all.append(cv_list)

e_factor: 2, s_factor: 0.6
134 134


<ipython-input-21-6195a5caad17>:18: DeprecationWarning: `np.int0` is a deprecated alias for `np.intp`.  (Deprecated NumPy 1.24)
  box=np.int0(cv2.boxPoints(rect))


87 87
85 85
126 126
149 149
132 132
e_factor: 2, s_factor: 1
134 134
87 87
85 85
126 126
149 149
132 132
e_factor: 2, s_factor: 1.4
134 129
87 83
85 77
126 118
149 145
132 126
e_factor: 4, s_factor: 0.6
124 123
85 85
84 84
120 118
144 142
128 128
e_factor: 4, s_factor: 1
124 123
85 85
84 84
120 118
144 142
128 128
e_factor: 4, s_factor: 1.4
124 123
85 85
84 84
120 118
144 142
128 128
e_factor: 6, s_factor: 0.6
122 118
85 84
84 83
118 113
141 135
127 126
e_factor: 6, s_factor: 1
122 118
85 84
84 83
118 113
141 135
127 126
e_factor: 6, s_factor: 1.4
122 118
85 84
84 83
118 113
141 135
127 126


In [ ]:
for idx, (test_image, _, grid, _) in enumerate(test_dataset):
    if idx == 6:
        break
    name = test_filenames[idx][:-4]
    micrograph = np.load(f"{IMAGE_DIR}/test/{name}.npy")
    label_path = f"{LABEL_DIR}/{name}.png"
    image = Image.open(label_path)
    label_image = np.array(image)

    cropped_micrograph = preprocess_and_crop(micrograph, crop_size=None)
    cropped_label_image = preprocess_and_crop(label_image, crop_size=None)

    #label_images = np.concatenate((label_images, [cropped_label_image]), axis=0)

    locations = labeled_particles[labeled_particles['image_name'] == name]
    _, ax = plt.subplots(figsize=(12, 12))
    plot_micrograph_and_labels(ax, cropped_micrograph, cropped_label_image, cv_list_all[0][idx])
    plt.show()

Output hidden; open in https://colab.research.google.com to view.

### TrackPy

In [ ]:
import trackpy as tp

In [ ]:
TrackParticleSize = RADIUS*2-1
curr_adpmass = 0
sep = [0.4, 0.7, 1]
#topn = 500
scale = [0.15, 0.25, 0.35]  # Scale factor (0.5 means reducing the size to half)

In [ ]:
label_images.shape

(6, 3838, 3710)

In [ ]:
tp_list_all = []
for e in scale:
    for s in sep:
        print(f"e_factor: {e}, s_factor: {s}")
        tp_list = []
        for img in label_images:
            output_image = img
            small_image = cv2.resize(output_image, None, fx=e, fy=e, interpolation=cv2.INTER_AREA)
            # Adjust parameters based on the scale
            sep_s = round(s * TrackParticleSize)+1
            small_sep = int(sep_s * e)
            small_diameter = int(TrackParticleSize * e)
            # Ensure small_diameter is odd
            if small_diameter % 2 == 0:
                small_diameter += 1
            coorTrack = tp.locate(small_image, diameter=small_diameter, minmass=curr_adpmass, separation=small_sep)
            #coorTrack.loc[:,'prob']=[output_image[getattr(coor,'y'),getattr(coor,'x')] for coor in np.floor(coorTrack[['x','y']]).astype('int').itertuples()]
            coorTrack['x'] *= (1/e)
            coorTrack['y'] *= (1/e)
            coords = coorTrack[['x', 'y']].values
            tp_list.append(coords)
            print(len(coords))
        tp_list_all.append(tp_list)

e_factor: 0.15, s_factor: 0.4
136
88
85
128
154
132
e_factor: 0.15, s_factor: 0.7
126
85
84
121
145
126
e_factor: 0.15, s_factor: 1
101
79
78
103
110
104
e_factor: 0.25, s_factor: 0.4
137
89
85
128
156
132
e_factor: 0.25, s_factor: 0.7
126
86
84
123
147
126
e_factor: 0.25, s_factor: 1
102
79
80
99
108
106
e_factor: 0.35, s_factor: 0.4
139
89
85
129
156
134
e_factor: 0.35, s_factor: 0.7
130
86
84
125
148
131
e_factor: 0.35, s_factor: 1
103
81
78
102
112
104


In [ ]:
for idx, (test_image, _, grid, _) in enumerate(test_dataset):
    if idx == 6:
        break
    name = test_filenames[idx][:-4]
    micrograph = np.load(f"{IMAGE_DIR}/test/{name}.npy")
    label_path = f"{LABEL_DIR}/{name}.png"
    image = Image.open(label_path)
    label_image = np.array(image)

    cropped_micrograph = preprocess_and_crop(micrograph, crop_size=None)
    cropped_label_image = preprocess_and_crop(label_image, crop_size=None)

    #label_images = np.concatenate((label_images, [cropped_label_image]), axis=0)

    locations = labeled_particles[labeled_particles['image_name'] == name]
    _, ax = plt.subplots(figsize=(12, 12))
    plot_micrograph_and_labels(ax, cropped_micrograph, cropped_label_image, tp_list_all[3][idx])
    plt.show()

Output hidden; open in https://colab.research.google.com to view.

### Nonmax

In [ ]:
import pycuda.driver as drv
from pycuda.compiler import SourceModule
import pycuda.autoinit  # Automatically initializes CUDA, creates a context which is needed

mod = SourceModule("""
__global__ void scoreGpu(float* heat,float*gaus, float*score, int sizex, int sizey, int psize)
{
    // 2D Thread ID (assuming that only *one* block will be executed)
    int tx=threadIdx.x;
    int ty=threadIdx.y;

    tx=blockIdx.x*blockDim.x+tx;
    ty=blockIdx.y*blockDim.y+ty;

    int tmp=psize/2;
    if((tx >= tmp) && (ty >= tmp) && (tx <= sizex-tmp) && (ty < sizey-tmp) ){
        int sx=tx-tmp;
        int sy=ty-tmp;
        heat=heat+sx*sizex+sy;
        //heat=heat+tx*sizex+ty;
        float sum=0;
        int sub=0;
        for(int i=0;i<psize;i++){
            for(int j=0;j<psize;j++){
                sum=sum+gaus[sub]*heat[j];
                sub=sub+1;
            }
            heat=heat+sizex;
        }
        score[tx*sizex+ty]=sum;

    }//end if
}
__global__ void getMax(float* score, float* list, int size, int sizex, int sizey, int numx){
    int tx=threadIdx.x;
    int ty=threadIdx.y;

    tx=blockIdx.x*blockDim.x+tx;
    ty=blockIdx.y*blockDim.y+ty;
    int cx=tx*size;
    int cy=ty*size;


    if(cx<sizex&&cy<sizey){
        int sx=size;
        int sy=size;

        if(cx+size>sizex){
            sx=sizex-cx;
        }
        if(cy+size>sizey){
            sy=sizey-cy;
        }
        score=score+cx*sizex+cy;
        //float max=6.805646932770577*(1000000000000000000000);
        float max=0;
        int maxx=0;
        int maxy=0;
        for(int i=0;i<sx;i++){
            for(int j=0;j<sy;j++){
                if(score[j]>=max){
                    max=score[j];
                    maxx=cx+i;
                    maxy=cy+j;
                }
            }
            score=score+sizex;
        }

        int sub=5*(tx*numx+ty);
        list[sub]=maxx;
        list[sub+1]=maxy;
        list[sub+2]=max;
        list[sub+3]=tx;
        list[sub+4]=ty;
    }
}

__global__ void getMax3(float* score, float* list, int psize, int sizex, int sizey, int numx,int num, int iter){
    int tx=threadIdx.x;
    int ty=threadIdx.y;

    tx=blockIdx.x*blockDim.x+tx;
    ty=blockIdx.y*blockDim.y+ty;
    int sub=(tx*numx+ty)*5;

    if (sub<num){
        int cx=list[sub]-psize/2;
        int cy=list[sub+1]-psize/2;

        float max=0;
        int maxx=0;
        int maxy=0;

        for(int i=0;i<iter;i++){

            if(cx<sizex&&cy<sizey&&cx>0&&cy>0){
                int sx=psize;
                int sy=psize;

                if(cx+sx>sizex){
                    sx=sizex-cx;
                }
                if(cy+sy>sizey){
                    sy=sizey-cy;
                }
                max=0;
                maxx=0;
                maxy=0;
                float* score0=score+cx*sizex+cy;
                for(int i=0;i<sx;i++){
                    for(int j=0;j<sy;j++){
                        if(score0[j]>=max){
                            max=score0[j];
                            maxx=cx+i;
                            maxy=cy+j;
                        }
                    }
                    score0=score0+sizex;
                }
                cx=maxx-psize/2;
                cy=maxy-psize/2;
            }
         }//end for

        list[sub]=maxx;
        list[sub+1]=maxy;
        list[sub+2]=max;
        list[sub+3]=tx;
        list[sub+4]=ty;
    }//end if num
}



__global__ void multiply_them(float *dest, float *a, float *b)
{
  const int i = threadIdx.x;
  dest[i] = a[i] * b[i];
}

""")

class item:
    def __init__(self,count=0,x=0 ,y=0 ):
        self.id=count
        self.minX = x
        self.minY=y
        self.maxX=x
        self.maxY=y
        self.totalN=0
        self.totalS=0
        self.x=x
        self.y=y
        self.p=0.1
    def __str__(self):
        line=str(self.id)+' '+str(self.minX)+' '+str(self.maxX)+' '+str(self.minY)+' '+str(self.maxY) \
                +' '+str(self.totalN)+' '+str(self.totalS)+' '+str(self.x)+' '+str(self.y)+' '+str(self.p)+'\n'
        return line

    def update(self,x,y):
        if(x<self.minX):
            self.minX=x
        elif(x>self.maxX):
            self.maxX=x

        if(y<self.minY):
            self.minY=y
        elif(y>self.maxY):
            self.maxY=y

        self.totalN=self.totalN+1

        return self

    def getS(self):
        width=self.maxX-self.minX+1
        length=self.maxY-self.minY+1
        self.totalS=width*length
        self.x=self.minX+width/2
        self.y=self.minY+width/2
        self.p=float(self.totalN)/self.totalS
        return self.totalS

def reshape(res,num):
    canList=[]
    for i in range(0,num):
        sub=i*5
        m=res[sub+2]
        if m!=0:
            candi=[int(res[sub]),int(res[sub+1]),m,int(res[sub+2]),int(res[sub+3])]
            canList.append(candi)
    print('convert')
    canList.sort(key=lambda x: x[2], reverse=True)

    #for i in range(0, 10):
    #    print(canList[i])
    return canList

def gaussian_kernel_2d_opencv(kernel_size = 3,sigma = 1):
    kx = cv2.getGaussianKernel(kernel_size,sigma)
    ky = cv2.getGaussianKernel(kernel_size,sigma)
    res=np.multiply(kx,np.transpose(ky))
    res=1-(res-np.max(res))/(-1*np.ptp(res))
    return res

def getOverlap(x,y,rx,ry,p):
    t=int(p/2)
    xmin=x-t
    xmax=x+t
    ymin=y-t
    ymax=y+t

    rxmin=rx-t
    rxmax=rx+t
    rymin=ry-t
    rymax=ry+t

    x1=abs(rxmin-xmax)
    x2=abs(rxmax-xmin)
    lx=min(x1,x2)

    y1=abs(rymin-ymax)
    y2=abs(rymax-ymin)
    ly=min(y1,y2)

    return lx*ly

def cleanCanList(canList,op1,psize):
    numCan=len(canList)
    op=op1*psize*psize
    tmp=[-psize,-psize,0]
    for i in range(0,len(canList)):
        candi=canList[i]
        if candi !=tmp:
            x=candi[0]
            y=candi[1]

            for j in range(i+1,len(canList)) :
                candi=canList[j]
                rx=candi[0]
                ry=candi[1]

                if abs(x-rx)<psize and abs(y-ry)<psize:
                    sOp=getOverlap(x,y,rx,ry,psize)
                    if sOp>op:
                        #print(rx,ry,sOp,op)
                        canList[j]=tmp #delete
                        (canList[i])[0]=0.8*x+0.2*rx
                        (canList[i])[1]=0.8*y+0.2*ry


    return [x for x in canList if x != tmp]

def writeStarHead(fstar):
    star=open(fstar,'w')
    star.write('data_\n\n')
    star.write('loop_\n')
    star.write('_rlnCoordinateX #1\n')
    star.write('_rlnCoordinateY #2\n')
    star.write('_rlnParticleSelectZScore  #3\n')
    return star

def writeCan(fstar,canList,sizex, sizey):
    fstar=writeStarHead(fstar)
    p=0
    l=len(canList)
    for i in range(0,l):
        c=canList[i]
        #line=str(c[0])+' '+str(c[1])+' '+str(c[2])+'\n'
        #x=c[1]
        #y=sizey-c[0]
        x=c[0]
        y=c[1]
        x0=y
        y0=x
        if(len(c)>2):
            p=c[2]

        line=str(x0)+' '+str(y0)+' '+str(p)+'\n'
        #print(line)
        fstar.write(line)
    fstar.close()

def reNorm(heatArr, nSep):
    ksize=5

    kernel = np.ones((ksize,ksize),np.uint8)
    img = cv2.erode(heatArr,kernel,iterations = nSep)
    #tmp=int(psize/4)
    #if tmp%2 ==0:
    #    tmp=tmp+1
    #heatArr= cv2.medianBlur(heatArr,tmp)
    #heatArr= cv2.medianBlur(heatArr,5)
    #heatArr= cv2.medianBlur(heatArr,5)
    #heatArr= cv2.medianBlur(heatArr,5)
    return heatArr

def reLev(heatArr, level):
    num=int(255/level)
    heatArr=heatArr/num
    heatArr=heatArr.astype(int)
    return heatArr

def draw(im, cx, cy, sx, sy):
    # Calculate rectangle coordinates
    startx = int(cx - sx / 2)
    starty = int(cy - sy / 2)  # Corrected to use sy
    endx = int(cx + sx / 2)
    endy = int(cy + sy / 2)

    # Calculate colors (simplified for demonstration; adjust as necessary)
    c = 1
    r = (c // 4) * 255
    g = ((c % 4) // 2) * 255
    b = (c % 2) * 255

    # Thickness of the rectangle border
    w = 2

    # Draw the rectangle
    cv2.rectangle(im, (startx, starty), (endx, endy), (b, g, r), w)

    return im

def checkScore(score):
    score0=np.zeros([score.shape[0], score.shape[1],3])
    print(score0.shape, score.shape)
    score0[:,:,0]=score[:,:]
    score0[:,:,1]=score[:,:]
    score0[:,:,2]=score[:,:]
    a=score0
    score0=(255*(a - np.max(a))/-np.ptp(a))
    for c in canList:
        x=int(c[0])
        y=int(c[1])
        sx=sy=psize
        score0=draw(score0,x,y,sx,sy)

In [ ]:
def pad_image(image_array):
    # Get the current height and width of the image
    height, width = image_array.shape

    # Find the smallest pixel value in the image to use as padding value
    min_pixel_value = np.min(image_array)

    # Determine the size of padding needed
    if height > width:
        diff = height - width
        padding = ((0, 0), (0, diff))  # Padding only to the right
    else:
        diff = width - height
        padding = ((0, diff), (0, 0))  # Padding only to the bottom

    # Apply padding, using the smallest pixel value found
    padded_image = np.pad(image_array, padding, mode='constant', constant_values=min_pixel_value)
    return padded_image

In [ ]:
#ratio=1
pnum=1000 #initial filtering, larger if candidate is more 2000 for 10017, should inverse proportional to pCans
pCans=[0.4, 0.5, 0.6] #the grid size smaller will generate more candidate
overlaps=[0, 0.3, 0.6] #allow for overlap, smaller will delete more
psize=RADIUS*2

# Nor affect too much
level=3
nSep=10
nIter=100

In [ ]:
nms_list_all = []
for e in pCans:
    for s in overlaps:
        pCan=e
        overlap=s
        print(f"e_factor: {e}, s_factor: {s}")
        nms_list = []
        for img in label_images:
            heatArr=normalize(pad_image(img))
            heatArr=reNorm(heatArr, nSep)
            heatArr=reLev(heatArr,level)
            gaus= gaussian_kernel_2d_opencv(kernel_size = psize,sigma = 0)

            # canList,score=scoreGpuLaunch(heatArr,gaus,psize, gsize, nIter, fscore)
            gsize=psize*pCan
            heat=heatArr.astype(np.float32)
            gaus=gaus.astype(np.float32)
            score=np.zeros(heat.shape).astype(np.float32)
            [sizex,sizey]=heat.shape
            sizex = int(sizex)
            sizey = int(sizey)
            psize = int(psize)
            gsize = int(gsize)

            func = mod.get_function("scoreGpu")

            tx=16
            ty=16
            bx=(sizex-1)//tx+1
            by=(sizey-1)//ty+1
            print('get score gpu',tx, ty, bx, by, gsize, psize)


            func(drv.In(heat), drv.In(gaus), drv.Out(score), np.int32(sizex), np.int32(sizey), np.int32(psize),
                block=(tx, ty, 1), grid=(int(bx), int(by)))

            # # Calculate necessary dimensions and convert all to int
            numx = int((sizex - 1) // gsize + 1)
            numy = int((sizey - 1) // gsize + 1)
            num = numx * numy
            tnum = num * 5

            # # Create the result array
            res = np.zeros(tnum).astype(np.float32)

            # # Block and grid dimensions
            bx = (numx - 1) // tx + 1
            by = (numy - 1) // ty + 1

            # Get the function from the module
            func = mod.get_function("getMax")

            # Call the function, ensure all parameters are the correct type
            func(drv.In(score), drv.Out(res), np.int32(gsize), np.int32(sizex), np.int32(sizey), np.int32(numx),
                block=(16, 16, 1), grid=(bx, by, 1))

            # Make sure all dimensions and sizes are properly cast to np.int32 to avoid ambiguity
            niter = np.int32(nIter)
            gsize = np.int32(gsize)
            sizex = np.int32(sizex)
            sizey = np.int32(sizey)
            numx = np.int32(numx)
            tnum = np.int32(num)

            print('get Max3', tx, ty, bx, by, niter, 'other : ', gsize, sizex, sizey, numx, tnum)
            func = mod.get_function("getMax3")
            # Ensuring the correct parameter order and type for the kernel invocation
            func(drv.In(score), drv.InOut(res), gsize, sizex, sizey, numx, tnum, niter,
                    block=(16, 16, 1), grid=(bx, by, 1))

            canList=reshape(res,num)

            print('Number of Particles before:', len(canList))
            if(len(canList)>pnum):
                canList=canList[:pnum]


            canList=cleanCanList(canList,overlap,psize)
            #canList=reCan(canList,ratio)
            print('Number of Particles:', len(canList))
            nms_list.append([(r[1],r[0]) for r in canList])
        nms_list_all.append(nms_list)
        print("\n")

e_factor: 0.4, s_factor: 0
get score gpu 16 16 240 240 61 154
get Max3 16 16 4 4 100 other :  61 3838 3838 63 3969
convert
Number of Particles before: 2662
Number of Particles: 57
get score gpu 16 16 240 240 61 154
get Max3 16 16 4 4 100 other :  61 3838 3838 63 3969
convert
Number of Particles before: 2216
Number of Particles: 42
get score gpu 16 16 240 240 61 154
get Max3 16 16 4 4 100 other :  61 3838 3838 63 3969
convert
Number of Particles before: 2181
Number of Particles: 45
get score gpu 16 16 240 240 61 154
get Max3 16 16 4 4 100 other :  61 3838 3838 63 3969
convert
Number of Particles before: 2709
Number of Particles: 64
get score gpu 16 16 240 240 61 154
get Max3 16 16 4 4 100 other :  61 3838 3838 63 3969
convert
Number of Particles before: 2863
Number of Particles: 71
get score gpu 16 16 240 240 61 154
get Max3 16 16 4 4 100 other :  61 3838 3838 63 3969
convert
Number of Particles before: 2779
Number of Particles: 54


e_factor: 0.4, s_factor: 0.3
get score gpu 16 16 240 

/usr/local/lib/python3.10/dist-packages/debugpy/_vendored/pydevd/_pydevd_bundle/pydevd_comm.py:658: UserWarning: device_allocation in out-of-thread context could not be cleaned up
  frame = None


get score gpu 16 16 240 240 77 154
get Max3 16 16 4 4 100 other :  77 3838 3838 50 2500
convert
Number of Particles before: 1734
Number of Particles: 77
get score gpu 16 16 240 240 77 154
get Max3 16 16 4 4 100 other :  77 3838 3838 50 2500
convert
Number of Particles before: 1478
Number of Particles: 72
get score gpu 16 16 240 240 77 154
get Max3 16 16 4 4 100 other :  77 3838 3838 50 2500
convert
Number of Particles before: 1424
Number of Particles: 60
get score gpu 16 16 240 240 77 154
get Max3 16 16 4 4 100 other :  77 3838 3838 50 2500
convert
Number of Particles before: 1773
Number of Particles: 87
get score gpu 16 16 240 240 77 154
get Max3 16 16 4 4 100 other :  77 3838 3838 50 2500
convert
Number of Particles before: 1843
Number of Particles: 80
get score gpu 16 16 240 240 77 154
get Max3 16 16 4 4 100 other :  77 3838 3838 50 2500
convert
Number of Particles before: 1786
Number of Particles: 76


e_factor: 0.5, s_factor: 0.3
get score gpu 16 16 240 240 77 154
get Max3 16 16 4

In [ ]:
for idx, (test_image, _, grid, _) in enumerate(test_dataset):
    if idx == 6:
        break
    name = test_filenames[idx][:-4]
    micrograph = np.load(f"{IMAGE_DIR}/test/{name}.npy")
    label_path = f"{LABEL_DIR}/{name}.png"
    image = Image.open(label_path)
    label_image = np.array(image)

    cropped_micrograph = preprocess_and_crop(micrograph, crop_size=None)
    cropped_label_image = preprocess_and_crop(label_image, crop_size=None)

    #label_images = np.concatenate((label_images, [cropped_label_image]), axis=0)

    locations = labeled_particles[labeled_particles['image_name'] == name]
    _, ax = plt.subplots(figsize=(12, 12))
    plot_micrograph_and_labels(ax, cropped_micrograph, cropped_label_image, nms_list_all[2][idx])
    plt.show()

Output hidden; open in https://colab.research.google.com to view.

### Score

In [ ]:
import torch
from torchvision.ops import box_iou

def centers_to_boxes(centers, width, height):
    centers = torch.tensor(centers, dtype=torch.float32)
    half_w, half_h = width / 2, height / 2
    boxes = torch.cat([centers - torch.tensor([half_w, half_h]),
                       centers + torch.tensor([half_w, half_h])], dim=1)
    return boxes

def calculate_iou_torchvision(boxes1, boxes2):
    return box_iou(boxes1, boxes2)

def evaluate_detection_raw(iou_matrix, iou_threshold=0.5):
    # Evaluate detection based on IoU matrix
    true_positives = 0
    matched_gt = set()
    matched_pred = set()

    for gt_idx in range(iou_matrix.size(0)):
        for pred_idx in range(iou_matrix.size(1)):
            if iou_matrix[gt_idx, pred_idx] >= iou_threshold and gt_idx not in matched_gt and pred_idx not in matched_pred:
                true_positives += 1
                matched_gt.add(gt_idx)
                matched_pred.add(pred_idx)

    precision = true_positives / iou_matrix.size(1) if iou_matrix.size(1) > 0 else 0
    recall = true_positives / iou_matrix.size(0) if iou_matrix.size(0) > 0 else 0
    return precision, recall

def evaluate_detection_raw_multiple(iou_matrices, iou_threshold=0.5):
    all_precision = []
    all_recall = []

    for iou_matrix in iou_matrices:
        true_positives = 0
        matched_gt = set()
        matched_pred = set()

        for gt_idx in range(iou_matrix.size(0)):
            for pred_idx in range(iou_matrix.size(1)):
                if iou_matrix[gt_idx, pred_idx] >= iou_threshold and gt_idx not in matched_gt and pred_idx not in matched_pred:
                    true_positives += 1
                    matched_gt.add(gt_idx)
                    matched_pred.add(pred_idx)

        precision = true_positives / iou_matrix.size(1) if iou_matrix.size(1) > 0 else 0
        recall = true_positives / iou_matrix.size(0) if iou_matrix.size(0) > 0 else 0

        all_precision.append(precision)
        all_recall.append(recall)

    # Calculate average precision and recall across all images
    avg_precision = sum(all_precision) / len(all_precision) if all_precision else 0
    avg_recall = sum(all_recall) / len(all_recall) if all_recall else 0

    return avg_precision, avg_recall

from itertools import product

def evaluate_detection_raw_multiple(iou_matrices, iou_threshold=0.5):
    all_precision = []
    all_recall = []

    for iou_matrix in iou_matrices:
        # Get the highest IoU value and corresponding prediction for each ground truth
        max_vals, best_preds = torch.max(iou_matrix, dim=1)
        valid = max_vals >= iou_threshold  # Filter by threshold

        # Indices of ground truths with valid predictions
        valid_gt_indices = torch.arange(iou_matrix.size(0))[valid]
        valid_pred_indices = best_preds[valid]

        # Create a tensor to track which predictions have been matched
        matched_preds = torch.zeros(iou_matrix.size(1), dtype=torch.bool)

        true_positives = 0
        for gt_idx, pred_idx in zip(valid_gt_indices, valid_pred_indices):
            if not matched_preds[pred_idx]:
                matched_preds[pred_idx] = True  # Mark this prediction as matched
                true_positives += 1

        total_predictions = iou_matrix.size(1)
        total_gts = iou_matrix.size(0)
        precision = true_positives / total_predictions if total_predictions > 0 else 0
        recall = true_positives / total_gts if total_gts > 0 else 0

        all_precision.append(precision)
        all_recall.append(recall)

    avg_precision = torch.mean(torch.tensor(all_precision))
    avg_recall = torch.mean(torch.tensor(all_recall))

    return avg_precision.item(), avg_recall.item()

def evaluate_detection(iou_matrix, scores, iou_threshold):
    sorted_indices = torch.argsort(scores, descending=True)
    sorted_iou_matrix = iou_matrix[:, sorted_indices]
    matched_gt = set()
    tp = 0
    precisions = []
    recalls = []
    #print(sorted_iou_matrix.shape)
    for i in range(sorted_iou_matrix.shape[1]):
        if sorted_iou_matrix[:, i].max() >= iou_threshold and sorted_iou_matrix[:, i].argmax() not in matched_gt:
            tp += 1
            matched_gt.add(sorted_iou_matrix[:, i].argmax())
        precision = tp / (i + 1)
        recall = tp / sorted_iou_matrix.shape[0]
        precisions.append(precision)
        recalls.append(recall)
    return torch.tensor(recalls), torch.tensor(precisions)

def calculate_ap(recalls, precisions):
    recalls = torch.cat([torch.tensor([0.0]), recalls, torch.tensor([1.0])])
    precisions = torch.cat([torch.tensor([0.0]), precisions, torch.tensor([0.0])])
    for i in range(precisions.size(0) - 2, -1, -1):
        precisions[i] = torch.max(precisions[i], precisions[i + 1])
    indices = torch.where(recalls[1:] != recalls[:-1])[0]
    ap = torch.sum((recalls[indices + 1] - recalls[indices]) * precisions[indices + 1])
    return ap

def calculate_mAP_per_image(boxes_gt, boxes_pred, scores, iou_thresholds=[0.5]):
    aps = []
    for iou_thresh in iou_thresholds:
        iou_matrix = box_iou(boxes_gt, boxes_pred)
        recalls, precisions = evaluate_detection(iou_matrix, scores, iou_thresh)
        ap = calculate_ap(recalls, precisions)
        aps.append(ap)
    return torch.mean(torch.tensor(aps))

def calculate_mAP_multiple_images(list_boxes_gt, list_boxes_pred, list_scores, iou_thresholds=[0.5]):
    # Expect lists of tensors for multiple images
    all_aps = []
    for boxes_gt, boxes_pred, scores in zip(list_boxes_gt, list_boxes_pred, list_scores):
        ap_per_image = calculate_mAP_per_image(boxes_gt, boxes_pred, scores, iou_thresholds)
        #print(ap_per_image)
        all_aps.append(ap_per_image)
    return torch.mean(torch.tensor(all_aps))

In [ ]:
# Assign a default confidence score of 1.0 to all predicted boxes
default_score = 1.0

In [ ]:
width = RADIUS*2
height = width

for i, cv_list in enumerate(cv_list_all):
    print(f"config {i}")
    iou_matrices = []
    gt_full = []
    pred_full = []
    for idx, gt in enumerate(gts):
        # Convert centers to boxes
        gt_boxes = centers_to_boxes(np.array(gt), width, height)
        pred_boxes = centers_to_boxes(np.array(cv_list[idx]), width, height)
        gt_full.append(gt_boxes)
        pred_full.append(pred_boxes)
        # Calculate IoU Matrix
        iou_matrix = calculate_iou_torchvision(gt_boxes, pred_boxes)
        iou_matrices.append(iou_matrix)

    scores = [torch.full((pred_boxes.shape[0],), default_score) for pred_boxes in pred_full]
    # Evaluate detection
    precision, recall = evaluate_detection_raw_multiple(iou_matrices, iou_threshold=0.5)
    print("Precision:", precision)
    print("Recall:", recall)
    # IoU thresholds for mAP calculation
    iou_thresholds = torch.arange(0.5, 1.0, 0.05)
    # Calculate mAP
    mAP_value = calculate_mAP_multiple_images(gt_full, pred_full, scores, iou_thresholds)
    print(f"Mean Average Precision (mAP): {mAP_value.item()}")

config 0
Precision: 0.9894132018089294
Recall: 0.9538059234619141
Mean Average Precision (mAP): 0.8425664901733398
config 1
Precision: 0.9894132018089294
Recall: 0.9538059234619141
Mean Average Precision (mAP): 0.8425664901733398
config 2
Precision: 0.9890542030334473
Recall: 0.9031693935394287
Mean Average Precision (mAP): 0.7968694567680359
config 3
Precision: 0.9642035365104675
Recall: 0.8903687596321106
Mean Average Precision (mAP): 0.8014078140258789
config 4
Precision: 0.9642035365104675
Recall: 0.8903687596321106
Mean Average Precision (mAP): 0.8014078140258789
config 5
Precision: 0.9642035365104675
Recall: 0.8903687596321106
Mean Average Precision (mAP): 0.8014078140258789
config 6
Precision: 0.9734747409820557
Recall: 0.8737974762916565
Mean Average Precision (mAP): 0.7998840808868408
config 7
Precision: 0.9734747409820557
Recall: 0.8737974762916565
Mean Average Precision (mAP): 0.7998840808868408
config 8
Precision: 0.9734747409820557
Recall: 0.8737974762916565
Mean Average P

In [ ]:
width = RADIUS*2
height = width

for i, tp_list in enumerate(tp_list_all):
    print(f"config {i}")
    iou_matrices = []
    gt_full = []
    pred_full = []
    for idx, gt in enumerate(gts):
        # Convert centers to boxes
        gt_boxes = centers_to_boxes(np.array(gt), width, height)
        pred_boxes = centers_to_boxes(np.array(tp_list[idx]), width, height)
        gt_full.append(gt_boxes)
        pred_full.append(pred_boxes)
        # Calculate IoU Matrix
        iou_matrix = calculate_iou_torchvision(gt_boxes, pred_boxes)
        iou_matrices.append(iou_matrix)

    scores = [torch.full((pred_boxes.shape[0],), default_score) for pred_boxes in pred_full]
    # Evaluate detection
    precision, recall = evaluate_detection_raw_multiple(iou_matrices, iou_threshold=0.5)
    print("Precision:", precision)
    print("Recall:", recall)
    # IoU thresholds for mAP calculation
    iou_thresholds = torch.arange(0.5, 1.0, 0.05)
    # Calculate mAP
    mAP_value = calculate_mAP_multiple_images(gt_full, pred_full, scores, iou_thresholds)
    print(f"Mean Average Precision (mAP): {mAP_value.item()}")

config 0
Precision: 1.0
Recall: 0.9758071899414062
Mean Average Precision (mAP): 0.8375613689422607
config 1
Precision: 1.0
Recall: 0.9307606816291809
Mean Average Precision (mAP): 0.8010441660881042
config 2
Precision: 1.0
Recall: 0.7917540073394775
Mean Average Precision (mAP): 0.682881772518158
config 3
Precision: 1.0
Recall: 0.9809291958808899
Mean Average Precision (mAP): 0.8890296816825867
config 4
Precision: 1.0
Recall: 0.9372732043266296
Mean Average Precision (mAP): 0.8509206771850586
config 5
Precision: 1.0
Recall: 0.7920481562614441
Mean Average Precision (mAP): 0.7211308479309082
config 6
Precision: 1.0
Recall: 0.9870278239250183
Mean Average Precision (mAP): 0.8775685429573059
config 7
Precision: 1.0
Recall: 0.9517531394958496
Mean Average Precision (mAP): 0.8351415991783142
config 8
Precision: 1.0
Recall: 0.7986195087432861
Mean Average Precision (mAP): 0.6990594267845154


In [ ]:
width = RADIUS*2
height = width

for i, nms_list in enumerate(nms_list_all):
    print(f"config {i}")
    iou_matrices = []
    gt_full = []
    pred_full = []
    for idx, gt in enumerate(gts):
        # Convert centers to boxes
        gt_boxes = centers_to_boxes(np.array(gt), width, height)
        pred_boxes = centers_to_boxes(np.array(nms_list[idx]), width, height)
        gt_full.append(gt_boxes)
        pred_full.append(pred_boxes)
        # Calculate IoU Matrix
        iou_matrix = calculate_iou_torchvision(gt_boxes, pred_boxes)
        iou_matrices.append(iou_matrix)

    scores = [torch.full((pred_boxes.shape[0],), default_score) for pred_boxes in pred_full]
    # Evaluate detection
    precision, recall = evaluate_detection_raw_multiple(iou_matrices, iou_threshold=0.5)
    print("Precision:", precision)
    print("Recall:", recall)
    # IoU thresholds for mAP calculation
    iou_thresholds = torch.arange(0.5, 1.0, 0.05)
    # Calculate mAP
    mAP_value = calculate_mAP_multiple_images(gt_full, pred_full, scores, iou_thresholds)
    print(f"Mean Average Precision (mAP): {mAP_value.item()}")

config 0
Precision: 0.990271270275116
Recall: 0.7282368540763855
Mean Average Precision (mAP): 0.5062408447265625
config 1
Precision: 1.0
Recall: 0.9372346997261047
Mean Average Precision (mAP): 0.7899429202079773
config 2
Precision: 1.0
Recall: 0.990081250667572
Mean Average Precision (mAP): 0.8621950745582581
config 3
Precision: 0.9769898056983948
Recall: 0.7636358141899109
Mean Average Precision (mAP): 0.4210287034511566
config 4
Precision: 0.9841406345367432
Recall: 0.9545422196388245
Mean Average Precision (mAP): 0.6624652743339539
config 5
Precision: 0.7398620247840881
Recall: 0.997652530670166
Mean Average Precision (mAP): 0.5887219309806824
config 6
Precision: 0.8989375233650208
Recall: 0.75177001953125
Mean Average Precision (mAP): 0.3185669481754303
config 7
Precision: 0.7929787039756775
Recall: 0.949131965637207
Mean Average Precision (mAP): 0.4462043344974518
config 8
Precision: 0.40347111225128174
Recall: 0.9963704943656921
Mean Average Precision (mAP): 0.3227379024028778


In [ ]:
# 500 filter
width = RADIUS*2
height = width

for i, nms_list in enumerate(nms_list_all):
    print(f"config {i}")
    iou_matrices = []
    gt_full = []
    pred_full = []
    for idx, gt in enumerate(gts):
        # Convert centers to boxes
        gt_boxes = centers_to_boxes(np.array(gt), width, height)
        pred_boxes = centers_to_boxes(np.array(nms_list[idx]), width, height)
        gt_full.append(gt_boxes)
        pred_full.append(pred_boxes)
        # Calculate IoU Matrix
        iou_matrix = calculate_iou_torchvision(gt_boxes, pred_boxes)
        iou_matrices.append(iou_matrix)

    scores = [torch.full((pred_boxes.shape[0],), default_score) for pred_boxes in pred_full]
    # Evaluate detection
    precision, recall = evaluate_detection_raw_multiple(iou_matrices, iou_threshold=0.5)
    print("Precision:", precision)
    print("Recall:", recall)
    # IoU thresholds for mAP calculation
    iou_thresholds = torch.arange(0.5, 1.0, 0.05)
    # Calculate mAP
    mAP_value = calculate_mAP_multiple_images(gt_full, pred_full, scores, iou_thresholds)
    print(f"Mean Average Precision (mAP): {mAP_value.item()}")

config 0
Precision: 0.9969136118888855
Recall: 0.45379534363746643
Mean Average Precision (mAP): 0.3477120101451874
config 1
Precision: 1.0
Recall: 0.5322409272193909
Mean Average Precision (mAP): 0.47332966327667236
config 2
Precision: 1.0
Recall: 0.5860583782196045
Mean Average Precision (mAP): 0.536098301410675
config 3
Precision: 0.9958913326263428
Recall: 0.6266944408416748
Mean Average Precision (mAP): 0.4728192389011383
config 4
Precision: 1.0
Recall: 0.7746624946594238
Mean Average Precision (mAP): 0.6885848045349121
config 5
Precision: 1.0
Recall: 0.8324839472770691
Mean Average Precision (mAP): 0.7563681602478027
config 6
Precision: 0.9945054650306702
Recall: 0.7157673835754395
Mean Average Precision (mAP): 0.5098094344139099
config 7
Precision: 1.0
Recall: 0.9018164277076721
Mean Average Precision (mAP): 0.7884635925292969
config 8
Precision: 1.0
Recall: 0.9602988362312317
Mean Average Precision (mAP): 0.8541183471679688
